# Klavis Sandbox

This notebook demonstrates how to run Klavis Sandbox environment and MCP Servers (remote + local) against a complicated task


In [ ]:
%pip install -q klavis python-dotenv langchain langchain-mcp-adapters langchain-anthropic

In [ ]:
import json
import os
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from klavis import Klavis, SandboxMcpServer

load_dotenv()

CURRENT_DIR = os.path.dirname(os.path.abspath("__file__")) if os.path.dirname(os.path.abspath("__file__")) else os.getcwd()
LOCAL_MCP_SERVERS_PATH = os.path.abspath(os.path.join(CURRENT_DIR, "..", "..", "mcp_servers", "local"))

klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

print(f"Current Directory: {CURRENT_DIR}")
print(f"MCP Servers Local Path: {LOCAL_MCP_SERVERS_PATH}")

### Step 1: Create Sandboxes environment for Gmail and Snowflake

In [25]:
# Create Gmail sandbox
gmail_sandbox = klavis_client.sandbox.create_sandbox(
    server_name=SandboxMcpServer.GMAIL,
)

snowflake_sandbox = klavis_client.sandbox.create_sandbox(
    server_name=SandboxMcpServer.SNOWFLAKE,
)

print(f"Gmail Sandbox: {gmail_sandbox}")
print(f"Snowflake Sandbox: {snowflake_sandbox}")

Gmail Sandbox: sandbox_id='1081a9e9-4615-4b64-b9f9-05b756bde058' server_url='https://gmail-mcp-server.klavis.ai/mcp/?instance_id=dc875dc9-38d8-4485-87cf-e0d14c42819c' server_name=<SandboxMcpServer.GMAIL: 'gmail'> status=<SandboxStatus.OCCUPIED: 'occupied'> message='Sandbox acquired successfully. Use the server_url to connect to the MCP server.'
Snowflake Sandbox: sandbox_id='b4444fdf-2f9c-44b5-89ca-eab8d05424e2' server_url='https://strata.klavis.ai/mcp/?instance_id=61dae4dc-70a3-4777-9343-eee162682211' server_name=<SandboxMcpServer.SNOWFLAKE: 'snowflake'> status=<SandboxStatus.OCCUPIED: 'occupied'> message='Sandbox acquired successfully. Use the server_url to connect to the MCP server.'


In [32]:
# check existing environment in gmail sandbox
response = klavis_client.sandbox.dump_sandbox(
    sandbox_id=gmail_sandbox.sandbox_id,
)

print(response)

ApiError: headers: {'content-type': 'application/json', 'x-cloud-trace-context': 'dd28afd524563658677d762173c5f770;o=1', 'date': 'Tue, 09 Dec 2025 21:28:10 GMT', 'server': 'Google Frontend', 'content-length': '90'}, status_code: 500, body: {'detail': 'Failed to dump sandbox: Snowflake account identifier is required in auth_data'}

In [26]:
# check existing environment in snowflake sandbox
response = klavis_client.sandbox.dump_sandbox(
    sandbox_id=snowflake_sandbox.sandbox_id,
)

print(json.dumps(response.data, indent=2))

{}


### Step 2: Initialize Snowflake Sandbox with Data

In [27]:
snowflake_json_path = os.path.join(CURRENT_DIR, "snowflake.json")

with open(snowflake_json_path, "r") as f:
    snowflake_data = json.load(f)
    
klavis_client.sandbox.initialize_sandbox(
    sandbox_id=snowflake_sandbox.sandbox_id,
    databases=snowflake_data["databases"],
)

InitializeSandboxResponse(sandbox_id='b4444fdf-2f9c-44b5-89ca-eab8d05424e2', status=<SandboxStatus.OCCUPIED: 'occupied'>, message='Created 8 total records across all object types', records_created={'databases': 1, 'schemas': 1, 'tables': 2, 'rows': 4})

In [28]:
# after loading data, check environment in snowflake sandbox
response = klavis_client.sandbox.dump_sandbox(
    sandbox_id=snowflake_sandbox.sandbox_id,
)

print(json.dumps(response.data, indent=2))

{
  "databases": [
    {
      "name": "SLA_MONITOR",
      "comment": "SLA monitoring and customer support ticket tracking system",
      "schemas": [
        {
          "name": "CORE",
          "tables": [
            {
              "name": "TICKETS",
              "columns": [
                {
                  "name": "ID",
                  "data_type": "NUMBER(38,0)",
                  "nullable": false
                },
                {
                  "name": "USER_ID",
                  "data_type": "NUMBER(38,0)",
                  "nullable": false
                },
                {
                  "name": "TITLE",
                  "data_type": "VARCHAR(500)",
                  "nullable": false
                },
                {
                  "name": "DESCRIPTION",
                  "data_type": "VARCHAR(16777216)",
                  "nullable": true
                },
                {
                  "name": "STATUS",
                  "data_type": "V

### Step 3: Setup MCP Client, AI Agent and kick off task

In [29]:
# Create MCP client with remote and local MCP Servers
mcp_client = MultiServerMCPClient({
    "snowflake": {
        "transport": "streamable_http",
        "url": snowflake_sandbox.server_url,
    },
    "gmail": {
        "transport": "streamable_http",
        "url": gmail_sandbox.server_url,
    },
    "filesystem": {
        "transport": "stdio",
        "command": "npx",
        "args": [
            "-y",
            os.path.join(LOCAL_MCP_SERVERS_PATH, "filesystem"),
            CURRENT_DIR
        ],
        "env": dict(os.environ)
    },
    "pdf-tools": {
        "transport": "stdio",
        "command": "uvx",
        "args": [
            "--from",
            os.path.join(LOCAL_MCP_SERVERS_PATH, "pdf-tools"),
            "pdf-tools-mcp",
            "--workspace_path",
            CURRENT_DIR,
            "--tempfile_dir",
            CURRENT_DIR
        ],
        "env": dict(os.environ)
    },
})

tools = await mcp_client.get_tools()
llm = ChatAnthropic(model="claude-sonnet-4-5-20250929")

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=(
        "You are an intelligent SLA Monitoring Agent. Your goal is to ensure compliance with Service Level Agreements (SLAs) and manage communications for breaches.\n"
        "You have access to below tools:\n"
        "1. Snowflake: To fetch user and ticket data.\n"
        "2. PDF/Filesystem: To read SLA manuals and email templates from files of the local filesystem.\n"
        "3. Gmail: to send emails if needed.\n\n"
        "General Workflow:\n"
        "1. Discovery: Explore the database to understand the data schema (users, tickets, etc.).\n"
        "2. SLA Extraction: Read the provided SLA manual (PDF) to determine response time limits for different service levels.\n"
        "3. Analysis: Compare the data against the SLA rules. Calculate elapsed time since creation.\n"
        "4. Action: For any breaches:\n"
        "   - Find the appropriate email template.\n"
        "   - Send a reminder to the manager.\n"
        "   - Send an apology to the user.\n\n"
        "Always verify the current date/time when calculating SLAs."
    ),
)

In [30]:
# print tools
for i, tool in enumerate(tools, 1):
    print(f"{i}. {tool.name}")

1. snowflake_list_databases
2. snowflake_create_database
3. snowflake_drop_database
4. snowflake_describe_database
5. snowflake_list_schemas
6. snowflake_create_schema
7. snowflake_drop_schema
8. snowflake_describe_schema
9. snowflake_list_tables
10. snowflake_describe_table
11. snowflake_preview_table
12. snowflake_get_table_row_count
13. snowflake_drop_table
14. snowflake_list_warehouses
15. snowflake_describe_warehouse
16. snowflake_create_warehouse
17. snowflake_drop_warehouse
18. snowflake_alter_warehouse
19. snowflake_execute_read_query
20. snowflake_execute_write_query
21. snowflake_get_account_info
22. snowflake_list_users
23. snowflake_list_roles
24. snowflake_describe_user
25. snowflake_describe_role
26. snowflake_list_grants_to_role
27. snowflake_get_current_account
28. snowflake_get_query_history
29. gmail_send_email
30. gmail_draft_email
31. gmail_read_email
32. gmail_search_emails
33. gmail_modify_email
34. gmail_delete_email
35. gmail_batch_modify_emails
36. gmail_batch_

In [31]:
user_message = (
    "Identify the tickets in the database that have exceeded the initial response time according to the relevant documentation, and send "
    "reminder emails, based on the templates mentioned in the manual, to the respective responsible managers, as well as apology emails to all involved users."
)

print("🚀 Starting agent...\n")

async for event in agent.astream_events(
    {"messages": [{"role": "user", "content": user_message}]},
    version="v2",
):
    kind = event["event"]
    
    if kind == "on_chat_model_stream":
        chunk = event["data"]["chunk"]
        # Try to extract content from the chunk
        if hasattr(chunk, "content"):
            content = chunk.content
            if content:
                # Handle both string and list content
                if isinstance(content, list):
                    for item in content:
                        if isinstance(item, dict) and "text" in item:
                            print(item["text"], end="", flush=True)
                        elif hasattr(item, "text"):
                            print(item.text, end="", flush=True)
                elif isinstance(content, str):
                    print(content, end="", flush=True)
    
    elif kind == "on_tool_start":
        tool_name = event["name"]
        tool_input = {k: v for k, v in event["data"].get("input", {}).items() 
                      if k not in ["runtime", "config", "stream_writer", "tool_call_id", "store"]}
        print(f"\n🔧 {tool_name}")
        if tool_input:
            print(f"   → {tool_input}")
    
    elif kind == "on_tool_end":
        print("\n")
        pass

print("\n\n✨ Done!")

🚀 Starting agent...

I'll help you identify SLA breaches and send the appropriate reminder and apology emails. Let me start by exploring the database structure and then reading the SLA documentation.
🔧 snowflake_list_databases

🔧 filesystem_list_allowed_directories




Good! Now let me explore the SLA_MONITOR database and the filesystem for relevant files.
🔧 snowflake_list_schemas
   → {'database': 'SLA_MONITOR'}

🔧 filesystem_list_directory
   → {'path': '/Users/zihaolin/src/klavis/examples/klavis-sandbox'}




Great! I found the `sla_manual.pdf`. Now let me check the CORE schema for tables and read the SLA manual.
🔧 snowflake_list_tables
   → {'database': 'SLA_MONITOR', 'schema': 'CORE'}

🔧 read_pdf
   → {'file_path': 'sla_manual.pdf'}




Perfect! Now let me examine the table structures and get the current data.
🔧 snowflake_describe_table
   → {'database': 'SLA_MONITOR', 'schema': 'CORE', 'table': 'TICKETS'}

🔧 snowflake_describe_table
   → {'database': 'SLA_MONITOR', 'schema': 'COR

ToolException: MCP error -32602: Output validation error: Invalid structured content for tool filesystem_directory_tree: [
  {
    "expected": "string",
    "code": "invalid_type",
    "path": [
      "content"
    ],
    "message": "Invalid input: expected string, received array"
  }
]

### Step 5: Cleanup - Delete Sandboxes

In [24]:
klavis_client.sandbox.delete_sandbox(
    server_name=SandboxMcpServer.GMAIL,
    sandbox_id=gmail_sandbox.sandbox_id,
)

klavis_client.sandbox.delete_sandbox(
    server_name=SandboxMcpServer.SNOWFLAKE,
    sandbox_id=snowflake_sandbox.sandbox_id,
)

ReleaseSandboxResponse(sandbox_id='b4444fdf-2f9c-44b5-89ca-eab8d05424e2', status=<SandboxStatus.IDLE: 'idle'>, message='Sandbox b4444fdf-2f9c-44b5-89ca-eab8d05424e2 released successfully and is now available for reuse')